In [12]:
pwd

'/pfs/data5/home/kit/lti/rk3078/JupyterNotebooks/00_Publication01_Unsupervised_Clustering/ML_for_Perovskite_Luminescence_Data'

In [13]:
# computation of the 5th channel of the dataset
# since the computation has to be conducted for each single pixel for each time step 
# for each patch of each substrate, this process takes a long time (multiple 10's of hours)
# e.g. 2min per patch for 1129 patches -> approx. 38 hours

In [14]:
import os
import glob
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
np.seterr(divide='ignore', invalid='ignore')

# function to determine the set of values in lookup talbe, which are closest to the 
# one generate by dividing the actual experimantal images of the different channels 
# through each other. The returned index can then be used to determine the wavelength
# of the peak of photolumnniscence (PL) spetrum (assuming the PL peak to have a gaussian
# shape)
def get_indices_of_closest_lookUp_value(lookUp_table_1, lookUp_table_2, exp_Value_1, exp_Value_2) :
    
    residual_squares_experimental_lookUp = np.sqrt(
                     np.square(abs(np.subtract(lookUp_table_1, exp_Value_1)))
                   + np.square(abs(np.subtract(lookUp_table_2, exp_Value_2))))

    
    flat_index_leastSquares = np.argmin(residual_squares_experimental_lookUp)

    index_wavelength = int(flat_index_leastSquares / len(FWHMs))

    return index_wavelength


In [15]:
# read-in look-up tables
# the look-up tables were generated in the following way: 
# gaussians with PL peaks between 700nm and 810nm (1nm increments)
# and full-width half-maxima (FWHM) between 20nm and 125nm (1nm increments)
# were simulated and multiplied by the transmission of the pathways of the
# different image channels (different spectral transmission of different 
# image channels and constant transmission of camera lens as well as
# camera sensor sensitivity)  before calcluating the integral of each 
# channel. For each simulated gaussian, the integrated signal of channel 1
# was divided by the integrated signal of channel 2 and saved in look-up 
# table 1 (same for channel 1 divide by channel 3 in look-up table 2).
# Experimentally, the same ratios of intensities of the different channels
# are computed from the acquired PL images. By finding the ratio-pair in the
# look-up table which is closest to the experimental ratios, an 
# approximation of the PL spectrum peak wavelength is found.

table1_filename = "LookUp_Table_1.txt"
table2_filename = "LookUp_Table_2.txt"

peakWavelengths = np.arange(700, 810 , 1)
FWHMs = np.arange(20, 125, 1)

if os.path.isfile("data_PLpeak_calculation"+os.sep + table1_filename) and os.path.isfile("data_PLpeak_calculation"+os.sep + table2_filename) :
    
    table1 = np.loadtxt("data_PLpeak_calculation"+os.sep + table1_filename,unpack=True)
    table2 = np.loadtxt("data_PLpeak_calculation"+os.sep + table2_filename,unpack=True)

    lookUp_table_1 = (np.transpose(table1)).flatten()
    lookUp_table_2 = (np.transpose(table2)).flatten()
    
    print("lookup tables loaded successfully")
else:
    print("error loading lookup tables")


lookup tables loaded successfully


In [17]:
dataset_path="/pfs/work7/workspace/scratch/rk3078-WorkSpace04a/ExperimentalData/insitu/01_croppedData/preprocessed_tryingout/"
counter=0

# iterate through the two folders of the subsets "test" and "train"
subset_info = sorted(glob.glob(dataset_path  +'*'))
for subset_iter in range(len(subset_info)):
    
    subset_path=subset_info[subset_iter]+"/"
    substrate_info = sorted(glob.glob(subset_path  +'A*'))
    
    # get list of all substrates in current subset and iterate through the substrates
    for substrate_iter in range(len(substrate_info)):

        substrate_path=substrate_info[substrate_iter]
        patch_info = sorted(glob.glob(substrate_path  +'/*'))
        
        # get list of all patches in current substrate and iterate through the patches
        for patch_iter in range(len(patch_info)):
            
            # load the numpy array
            patch_path=patch_info[patch_iter]
            patch_4_channels=np.load(patch_path).astype('float')

            # check, if the loaded data of the current patch has 4 channels.
            # if yes, continue with computation, if no, then the 5th channel
            # has already been added for this patch before -> continue with 
            # with next patch
            if patch_4_channels.shape[1]==4:
                
                # for all channels, the first image is used as background without PL signal to be subtracted from each following image
                lp725_channel_background = patch_4_channels[0,1,:,:]
                lp780_channel_background = patch_4_channels[0,2,:,:]
                sp775_channel_background = patch_4_channels[0,3,:,:]
                
                # the data of the 3 PL channels is extracted
                lp725_channel = patch_4_channels[:,1,:,:]
                lp780_channel = patch_4_channels[:,2,:,:]
                sp775_channel = patch_4_channels[:,3,:,:]
                
                
                # for the 3 channels, subtract channel background
                lp725_channel_backgroundCorrected = lp725_channel - lp725_channel_background
                lp780_channel_backgroundCorrected = lp780_channel - lp780_channel_background
                sp775_channel_backgroundCorrected = sp775_channel - sp775_channel_background

                # if background correction leads to value smaller than 0, set it to 0 
                # (only the case in the beginning when there is only noise and no PL signal yet)
                lp725_channel_backgroundCorrected[lp725_channel_backgroundCorrected<0] =  0  
                lp780_channel_backgroundCorrected[lp780_channel_backgroundCorrected<0] =  0    
                sp775_channel_backgroundCorrected[sp775_channel_backgroundCorrected<0] =  0
                    
                    

                # dimensions of the images
                height=65 
                width=56
                # container used for data to be added as 5th channel
                plPeakData = np.zeros((719,height,width))

                # iterate through all time steps
                for timeframe_iter in tqdm(range(719)):
                    
                    # for the 3 channels, get the image of the current time step                
                    lp725_channel_currentFrame_backgroundCorrected = lp725_channel_backgroundCorrected[timeframe_iter,:,:]
                    lp780_channel_currentFrame_backgroundCorrected = lp780_channel_backgroundCorrected[timeframe_iter,:,:]
                    sp775_channel_currentFrame_backgroundCorrected = sp775_channel_backgroundCorrected[timeframe_iter,:,:]
                    
                    # generate experimental ratios for filter1/filter2 and filter1/filter3 
                    image_channel_ratio_1 = np.divide(lp725_channel_currentFrame_backgroundCorrected,lp780_channel_currentFrame_backgroundCorrected).flatten();
                    image_channel_ratio_2 = np.divide(lp725_channel_currentFrame_backgroundCorrected,sp775_channel_currentFrame_backgroundCorrected).flatten();
                    
                    # container used for computed wavelengths
                    resultWavelengthsArray = np.array([0.0]*height*width).flatten();
                    
                    # iterate through all pixels in flattened image
                    for pixelNumber in range(len(resultWavelengthsArray)):
                        
                        # get experimental ratios (filter1/filter2 and filter1/filter3) of the current pixel
                        exp_Value_1 = image_channel_ratio_1[pixelNumber];
                        exp_Value_2 = image_channel_ratio_2[pixelNumber];
                        
                        # get index of simulated gaussian which is closest to the expermental values
                        index_wavelength = get_indices_of_closest_lookUp_value(lookUp_table_1, lookUp_table_2, exp_Value_1, exp_Value_2)

                        # get corresponding wavelength of computed index 
                        resultWavelengthsArray[pixelNumber] = peakWavelengths[index_wavelength]

                    # bring flatten wavelength array back to shape of the image
                    resultWavelengthsArray=np.reshape(resultWavelengthsArray,(height, width))

                    # add computed wavelength image to container at current time step
                    plPeakData[timeframe_iter,:,:]=resultWavelengthsArray



                # add computed 5th channel to the original 4 channels
                patch_5_channels = np.concatenate((patch_4_channels, np.expand_dims(plPeakData, axis=(1))),axis=1)


                # save the data by over-writing the 4-channel data of the patch with the 5-channel data
                if True:    
                    np.save(patch_path, patch_5_channels.astype('uint16'))
                    print("for ", counter+1, "/1129 patches successfully added 5th channel, current: ", '/'.join(patch_path.split("/")[-3:]))
                    counter+=1


            else:
                counter+=1                 


        print('/'.join(substrate_path.split("/")[-2:]), "done")
                
        
            

test/ACE done
test/ACG done
test/ACI done
test/ADB done
test/ADF done
test/ADH done
test/ADT done
test/AEA done
test/AEB done
test/AEE done


100%|██████████| 719/719 [01:49<00:00,  6.56it/s]


for  318 /1129 patches successfully added 5th channel, current:  test/AEH/81.npy


100%|██████████| 719/719 [01:49<00:00,  6.57it/s]


for  319 /1129 patches successfully added 5th channel, current:  test/AEH/82.npy


...

for  327 /1129 patches successfully added 5th channel, current:  test/AEL/22.npy


100%|██████████| 719/719 [01:49<00:00,  6.55it/s]


for  328 /1129 patches successfully added 5th channel, current:  test/AEL/23.npy


 39%|███▉      | 279/719 [00:41<01:05,  6.73it/s]


KeyboardInterrupt: 